<a href="https://colab.research.google.com/github/nav-in27/CUSTOMER-CHURN-PREDICTION-USING-AUTOGLUON/blob/main/Customer_churn_prediction_using_autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MODULE 1: Installation & Environment Setup

In [1]:
!pip install -q pandas==2.2.2
!pip install -q "autogluon.tabular[all]" scikit-learn joblib shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 791.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip -q install autogluon.tabular==1.1.0 scikit-learn pandas numpy joblib shap

ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python >=3.6, <3.9; 0.1.0b20210207 Requires-Python >=3.6, <3.8; 0.1.0b20210208 Requires-Python >=3.6, <3.8; 0.1.0b20210209 Requires-Python >=3.6, <3.8; 0.1.0b20210210 Requires-Python >=3.6, <3.8; 0.1.0b20210211 Requires-Python >=3.6, <3.8; 0.1.0b20210212 Requires-Python >=3.6, <3.8; 0.1.0b20210213 Requires-Python >=3.6, <3.8; 0.1.0b20210214 Requires-Python >=3.6, <3.8; 0.1.0b20210215 Requires-Python >=3.6, <3.8; 0.1.0b20210216 Requires-Python >=3.6, <3.8; 0.1.0b20210217 Requires-Python >=3.6, <3.8; 0.1.0b20210218 Requires-Python >=3.6, <3.8; 0.1.0b20210219 Requires-Python >=3.6, <3.8; 0.1.0b20210220 Requires-Python >=3.6, <3.8; 0.1.0b20210221 Requires-Python >=3.6, <3.8; 0.1.0b20210222 Requires-Python >=3.6, <3.8; 0.1.0b20210223 Requires-Python >=3.6, <3.8; 0.1.0b20210224 Requires-Python >=3.6, <3.8; 0.1.0b20210225 Requires-Python >=3.6, <3.9; 0.1.0b20210226 Requires-Python >=3.6, <3.9; 0.1.0b

In [3]:
import pandas as pd, numpy as np, warnings, joblib, time, json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, f1_score
from autogluon.tabular import TabularPredictor
warnings.filterwarnings("ignore")

MODULE 2: Data Loading & Preprocessing




In [4]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

Saving telecom_customer_churn.csv to telecom_customer_churn.csv


In [5]:
df.columns = df.columns.str.strip().str.lower()
target_candidates = [c for c in df.columns if 'churn' in c]
if not target_candidates:
    raise ValueError("No 'churn' column found! Available columns: " + ", ".join(df.columns))
target_col = target_candidates[0]
print(f"✓ Target column detected: '{target_col}'")


✓ Target column detected: 'churn category'


In [6]:
unique_vals = df[target_col].dropna().unique()
print(f"  Target values: {unique_vals}")
if df[target_col].dtype == 'object':

    df[target_col] = df[target_col].str.strip().str.lower()

  Target values: ['Competitor' 'Dissatisfaction' 'Other' 'Price' 'Attitude']


In [7]:
for col in df.columns:
    if col != target_col and ('charge' in col or 'tenure' in col or 'month' in col or 'age' in col):
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [8]:
initial_len = len(df)
df = df.dropna(subset=[target_col])
df = df.dropna(axis=0, how='all')
print(f"✓ Cleaned data: {len(df)} rows ({initial_len - len(df)} removed)")


✓ Cleaned data: 1869 rows (5174 removed)


MODULE 3: Problem Understanding & Data Splitting

In [9]:
class_counts = df[target_col].value_counts().sort_index()
print(f" Class distribution:\n{class_counts}")
n_classes = len(class_counts)
problem_type = 'binary' if n_classes == 2 else 'multiclass'
print(f"   Detected {problem_type} problem ({n_classes} classes)")


 Class distribution:
churn category
attitude           314
competitor         841
dissatisfaction    321
other              182
price              211
Name: count, dtype: int64
   Detected multiclass problem (5 classes)


In [10]:
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df[target_col])
print(f"✓ Train: {len(train)} | Test: {len(test)}")

✓ Train: 1495 | Test: 374


MODULE 4: Model Building Using AutoGluon

In [11]:
print(f"\n Training AutoGluon for {problem_type} classification...")
start = time.time()
predictor = TabularPredictor(
    label=target_col,
    eval_metric='roc_auc' if problem_type == 'binary' else 'accuracy',
    problem_type=problem_type,
    path='best_churn_model'
).fit(
    train,
    time_limit=600,
    presets='best_quality',
    verbosity=2
)
training_time = (time.time() - start) / 60
print(f"✓ Training finished in {training_time:.1f} min")


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.09 GB / 12.67 GB (87.5%)
Disk Space Avail:   68.52 GB / 112.64 GB (60.8%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdou


 Training AutoGluon for multiclass classification...


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-11-23 11:21:42,038	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/content/best_churn_model/ds_sub_fit/sub_fit_ho"
(_dystack pid=2325) Running DyStack sub-fit ...
(_dystack pid=2325) Beginning AutoGluon training ... Time limit = 131s
(_dystack pid=2325) AutoGluon will save models to "/content/best_churn_model/ds_sub_fit/sub_fit_ho"
(_dystack pid=2325) Train Data Rows:    1328
(_dystack pid=2325) Train Data Columns: 37
(_dystack pid=2325) Label Column:       churn category
(_dystack pid=2325) Problem Type:       multiclass
(_dystack pid=2325) Preprocessing data ...
(_dystack pid=2325) Train Data Class Count: 5
(_dystack pid=2325) Using Feature Generators to preprocess the data ...
(_dystack pid=2325) Fitting AutoMLPipelin

✓ Training finished in 10.0 min


MODULE 5: Model Evaluation

In [12]:
print("\n Evaluating on test set...")
test_data = test.drop(columns=[target_col])
preds = predictor.predict(test_data)


if problem_type == 'binary':
    proba_df = predictor.predict_proba(test_data)
    if isinstance(proba_df, pd.DataFrame):
        proba = proba_df[1].values if 1 in proba_df.columns else proba_df.iloc[:, 1].values
    else:
        proba = proba_df
    roc = roc_auc_score(test[target_col], proba)
    f1 = f1_score(test[target_col], preds)
    print(f"\n{'='*50}")
    print(f"Final Scores → ROC-AUC: {roc:.3f}  |  F1: {f1:.3f}")
    print(f"{'='*50}\n")
else:

    from sklearn.metrics import accuracy_score
    acc = accuracy_score(test[target_col], preds)
    print(f"\n{'='*50}")
    print(f"Final Accuracy: {acc:.3f}")
    print(f"{'='*50}\n")

print(classification_report(test[target_col], preds, digits=3))


 Evaluating on test set...

Final Accuracy: 1.000

                 precision    recall  f1-score   support

       attitude      1.000     1.000     1.000        63
     competitor      1.000     1.000     1.000       168
dissatisfaction      1.000     1.000     1.000        64
          other      1.000     1.000     1.000        37
          price      1.000     1.000     1.000        42

       accuracy                          1.000       374
      macro avg      1.000     1.000     1.000       374
   weighted avg      1.000     1.000     1.000       374



MODULE 6: Feature Importance & Model Insights

In [13]:
print("\n Top 3 Churn Drivers:")
try:
    importance = predictor.feature_importance(test)
    top3 = importance.head(3)
    for i, (feat, score) in enumerate(top3.iterrows(), 1):
        print(f"  {i}. {feat:30s} (importance: {score['importance']:.4f})")
except Exception as e:
    print(f"  Unable to compute feature importance: {e}")

print("\n Pipeline complete!")

These features in provided data are not utilized by the predictor and will be ignored: ['customer id', 'customer status']
Computing feature importance via permutation shuffling for 35 features using 374 rows with 5 shuffle sets...
	9.49s	= Expected runtime (1.9s per shuffle set)



 Top 3 Churn Drivers:


	1.77s	= Actual runtime (Completed 5 of 5 shuffle sets)


  1. churn reason                   (importance: 0.7187)
  2. streaming tv                   (importance: 0.0016)
  3. gender                         (importance: 0.0000)

 Pipeline complete!


MODULE 7: USER INPUT PREDICTION

In [15]:


import pandas as pd
from autogluon.tabular import TabularPredictor

# Load trained model
predictor = TabularPredictor.load("best_churn_model")

print(" Model loaded successfully!")


all_original_features = [col for col in train.columns if col not in [target_col, 'customer id', 'customer status']]

user_requested_features = [
    "tenure in months",
    "number of dependents",
    "number of referrals",
    "monthly charge",
    "total extra data charges"
]


top_features = [f for f in user_requested_features if f in all_original_features]

if top_features:
    print("\nFeatures for user input:")
    for f in top_features:
        print(f" • {f}")
else:
    print("Warning: None of the requested features were found in the dataset.")
    print("Using fallback features (if any). You may need to check the exact column names.")
    top_features = all_original_features[:5]
    for f in top_features:
        print(f" • {f}")



user_input_dict = {}
print("\nEnter values for the following fields:")

# Create a reference row from the training data to use as default values
# For robustness, using mode for categorical and mean for numerical columns.
default_values_series = train[all_original_features].apply(lambda x: x.mode()[0] if x.dtype == 'object' else x.mean())
default_values = default_values_series.to_dict()

for feature in all_original_features:
    if feature in top_features:
        # Prompt user for input for top features
        current_default = default_values.get(feature, 'N/A')
        user_value = input(f"Enter value for '{feature}' (default: '{current_default}'): ")
        if user_value:
            user_input_dict[feature] = user_value
        else:
            user_input_dict[feature] = default_values[feature]  # Use default if user leaves blank
    else:
        # For other features, use default values from the training data
        user_input_dict[feature] = default_values[feature]

# Convert to dataframe
input_df = pd.DataFrame([user_input_dict])

# Convert numeric-looking inputs (for the entire input_df)
for col in input_df.columns:
    # safe conversion: if it can't be converted, it will remain as is
    input_df[col] = pd.to_numeric(input_df[col], errors='ignore')


prediction = predictor.predict(input_df)[0]

# Predict probability
try:
    proba = predictor.predict_proba(input_df)
    if problem_type == 'binary':
        churn_prob = proba.iloc[0].iloc[1]  # probability of the positive class
    else:
        # For multiclass, get probability of the predicted class
        churn_prob = proba.iloc[0][prediction]
except Exception as e:
    churn_prob = None
    print(f"Warning: Could not get prediction probabilities. Error: {e}")


print("\n==================== RESULT ====================")
print(f"Predicted Churn: {prediction}")

if churn_prob is not None:
    if problem_type == 'binary':
        print(f"Churn Probability (Positive Class): {churn_prob:.3f}")
    else:
        print(f"Probability of '{prediction}': {churn_prob:.3f}")
else:
    print("Probability not available.")

print("================================================\n")

 Model loaded successfully!

Features for user input:
 • tenure in months
 • number of dependents
 • number of referrals
 • monthly charge
 • total extra data charges

Enter values for the following fields:
Enter value for 'number of dependents' (default: '0.12575250836120402'): 1.21
Enter value for 'number of referrals' (default: '0.5197324414715719'): 2
Enter value for 'tenure in months' (default: '17.8247491638796'): 20
Enter value for 'monthly charge' (default: '73.33819397993311'): 150
Enter value for 'total extra data charges' (default: '6.60200668896321'): 7

==================== RESULT ====================
Predicted Churn: competitor
Probability of 'competitor': 0.340

